In [52]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames

In [53]:
#load the data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/data/may9/capacity.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/data/may9/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/data/may9/LCA_model.csv",header=1,delim=",") |> DataFrame 
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/data/may9/emission_sink.csv",header=1,delim=",") |> DataFrame;

In [54]:
cell_demand = 0.001*226.0*(1e6)               # annual demand of Li battery for tesla (2M EV/yr, 3 NMC111 pack/EV, 164.98 kg/pack)

global_sink = 1.099e10                        # global CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
es_ratio = global_sink/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
EF_input = LCA_model[!,"EF (kg CO2/material)"]
process = LCA_model[!,"process"]
countries = capacity[!,"Country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
input_amount = LCA_model[!,"input (material/kg battery)"] * cell_demand;

In [55]:
LCA_model

Row,process,input (material/kg battery),EF (kg CO2/material),scaler
,String15,Float64,Float64,Float64
1,Li,0.11,7.5,0.383
2,Co,0.154,23.6,0.536
3,Mn,0.15,0.709,0.522
4,Ni,0.154,7.32,0.535
5,cathode,0.287,6.6,0.396
6,graphite,0.16,0.0679,0.221
7,PP,0.0116,2.21,0.016
8,PE,0.0029,2.4,0.004
9,Cu,0.0689,1.11,0.095


In [107]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15]

1.0

In [108]:
input_amount = []
input_amount = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount

15-element Vector{Float64}:
  24850.801799999997
  34778.1456
  33869.7612
  34713.261
  64884.600000000006
  36210.85
   2621.6
    655.4
  15565.749999999998
  30148.399999999998
 163850.0
   1130.0
    904.0
 226000.0
 226000.0

----

In [124]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [125]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0);   # y[i,j,k] ship product k from i to j 

In [126]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end

In [127]:
# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end

In [128]:
# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:40 if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end

In [129]:
for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

In [130]:
# @constraint(model, sum(x[i,9] for i in 1:ncty) == input_amount[9])

In [131]:
# # final demand constraint
# @constraint(model, sum(x[i, mkt_proc] for i in 1:ncty) == cell_demand);

In [132]:
# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand);

In [133]:
# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end


In [134]:
# 1. production emission
val_x = zeros(ncty, nproc)

for k in 1:nproc
    for i in 1:ncty
        val_x[i,k] = EF_input[k] * (1 - sink_c[i]/emission_c[i] - es_ratio)
    end
end

proc_emi = sum(x .* val_x);

In [135]:
# 2. transpotration emission
trans_emi = 0

for i in 1:ncty
    for j in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        trans_emi += amount * distance[!, 2:end][i,j] * EF_trans
    end
end

In [136]:
# # 2. transpotration emission
# trans_emi = 0
# for k in 1:nproc
#     for i in 1:ncty
#         for j in 1:ncty
#             trans_emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans;
#         end
#     end
# end

In [137]:
@objective(model, Min, proc_emi+trans_emi);

In [138]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3335 rows, 24600 columns and 50320 nonzeros
Model fingerprint: 0x0313f911
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [2e-03, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 3150 rows and 23724 columns
Presolve time: 0.02s
Presolved: 185 rows, 876 columns, 1716 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.010e+02
 Factor NZ  : 2.486e+03
 Factor Ops : 3.829e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time

In [139]:
optx = copy(JuMP.value.(x))
opty = copy(JuMP.value.(y));

In [140]:
optx

40×15 Matrix{Float64}:
  3010.8      0.0    0.0      0.0  …     0.0    0.0       0.0       0.0
     0.0   7400.0    0.0      0.0        0.0    0.0       0.0       0.0
     0.0      0.0    0.0      0.0        0.0    0.0       0.0       0.0
     0.0      0.0    0.0      0.0        0.0    0.0       0.0       0.0
  1800.0      0.0    0.0      0.0        0.0    0.0       0.0       0.0
  1540.0   4440.0    0.0  34713.3  …     0.0    0.0       0.0       0.0
 18500.0      0.0    0.0      0.0        0.0  904.0       0.0       0.0
     0.0      0.0    0.0      0.0        0.0    0.0       0.0       0.0
     0.0      0.0    0.0      0.0        0.0    0.0       0.0       0.0
     0.0  10400.0    0.0      0.0        0.0    0.0       0.0       0.0
     0.0   3500.0    0.0      0.0  …     0.0    0.0       0.0       0.0
     0.0   1380.0    0.0      0.0        0.0    0.0       0.0       0.0
     0.0      0.0    0.0      0.0        0.0    0.0       0.0       0.0
     ⋮                             ⋱     

In [141]:
using Tables
res = Tables.table(optx[1:end, 1:end]);
CSV.write("/Users/bourg/Desktop/x_matrix3.csv", res) 

"/Users/bourg/Desktop/x_matrix3.csv"

In [62]:
res_y02 = Tables.table(opty[1:end, 1:end, 02])
CSV.write("/Users/bourg/Desktop/y02_matrix.csv", res_y02) 

"/Users/bourg/Desktop/y02_matrix.csv"